<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-단독다가구" data-toc-modified-id="1.-단독다가구-1">1. 단독다가구</a></span><ul class="toc-item"><li><span><a href="#1.1-초역세권/역세권,-전철역명(500m)-컬럼-추가" data-toc-modified-id="1.1-초역세권/역세권,-전철역명(500m)-컬럼-추가-1.1">1.1 초역세권/역세권, 전철역명(500m) 컬럼 추가</a></span></li><li><span><a href="#1.2-환승역-컬럼-추가" data-toc-modified-id="1.2-환승역-컬럼-추가-1.2">1.2 환승역 컬럼 추가</a></span></li></ul></li><li><span><a href="#2.-연립" data-toc-modified-id="2.-연립-2">2. 연립</a></span><ul class="toc-item"><li><span><a href="#2.1-초역세권/역세권,-전철역명(500m)-컬럼-추가" data-toc-modified-id="2.1-초역세권/역세권,-전철역명(500m)-컬럼-추가-2.1">2.1 초역세권/역세권, 전철역명(500m) 컬럼 추가</a></span></li><li><span><a href="#2.2-환승역-컬럼-추가" data-toc-modified-id="2.2-환승역-컬럼-추가-2.2">2.2 환승역 컬럼 추가</a></span></li></ul></li><li><span><a href="#3.-오피스텔" data-toc-modified-id="3.-오피스텔-3">3. 오피스텔</a></span><ul class="toc-item"><li><span><a href="#3.1-초역세권/역세권,-전철역명(500m)-컬럼-추가" data-toc-modified-id="3.1-초역세권/역세권,-전철역명(500m)-컬럼-추가-3.1">3.1 초역세권/역세권, 전철역명(500m) 컬럼 추가</a></span></li><li><span><a href="#3.2-환승역-컬럼-추가" data-toc-modified-id="3.2-환승역-컬럼-추가-3.2">3.2 환승역 컬럼 추가</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
from haversine import haversine
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

## 1. 단독다가구

In [2]:
data = pd.read_csv('data/단독다가구_위도경도.csv')
print(data.shape)
data.head()

(12342, 3)


,도로명,위도,경도
0,논현로6길,37.473561,127.050895
1,논현로18길,37.477885,127.046338
2,개포로20길,37.477040,127.048356
3,선릉로12길,37.483899,127.060991
4,논현로8길,37.474467,127.051458


In [3]:
subway = pd.read_csv('data/지하철_위도경도.csv')
print(subway.shape)
subway.head()

(391, 10)


,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소,위도,경도,환승역
0,2622,망원역,Mangwon,6호선,621,망원역 6호선,서울 마포구 월드컵로 지하 77,37.556057,126.910034,0
1,2623,합정역,Hapjeong,6호선,622,합정역 6호선,서울 마포구 양화로 지하 45,37.549123,126.913545,1
2,2624,상수역,Sangsu,6호선,623,상수역 6호선,서울 마포구 독막로 지하 85,37.547772,126.922410,0
3,2625,광흥창역,Gwangheungchang,6호선,624,광흥창역 6호선,서울 마포구 독막로 지하 165,37.547486,126.931939,0
4,2626,대흥역,Daeheung,6호선,625,대흥역 6호선,서울 마포구 대흥로 지하 85,37.547668,126.942465,0


In [4]:
# sample
start = (float(data['위도'][0]), float(data['경도'][0]))
goal = (float(subway['위도'][0]), float(subway['경도'][0]))
haversine(start, goal, unit='m')   # 단위 : m

15443.45635081588

### 1.1 초역세권/역세권, 전철역명(500m) 컬럼 추가

- 초역세권/역세권 : 역세권X - 0, 역세권(500m이내) - 1, 초역세권(300m이내) - 2

In [5]:
data['초역세권/역세권'] = 0
data['전철역명(500m)'] = 0

In [6]:
for i in tqdm(range(len(data))):
    time.sleep(0.1)
    list1 = []
    for j in range(len(subway)):
        start = (float(data['위도'][i]), float(data['경도'][i]))
        goal = (float(subway['위도'][j]), float(subway['경도'][j]))
        distance = haversine(start, goal, unit='m')
        if distance <= 500:
            list1.append(subway['검색키워드'][j])
            if distance <= 300:
                data['초역세권/역세권'][i] = 2
            elif data['초역세권/역세권'][i] != 2:
                data['초역세권/역세권'][i] = 1
    data['전철역명(500m)'][i] = np.array(list1, dtype=str)

100%|████████████████████████████████████████████████████████████████████████████| 12342/12342 [26:38<00:00,  7.72it/s]


In [7]:
data['초역세권/역세권'].value_counts()

0    5863
1    3772
2    2707
Name: 초역세권/역세권, dtype: int64

In [8]:
data['전철역명(500m)']

0                                               []
1                                               []
2                                               []
3                                        [구룡역 분당선]
4                                               []
                           ...                    
12337               [동묘앞역 6호선, 동대문역 4호선, 동대문역 1호선]
12338                                           []
12339                        [동대문역 4호선, 종로5가역 1호선]
12340    [을지로3가역 2호선, 충무로역 4호선, 명동역 4호선, 충무로역 3호선]
12341                                    [중화역 7호선]
Name: 전철역명(500m), Length: 12342, dtype: object

### 1.2 환승역 컬럼 추가

In [9]:
data['환승역'] = 0

In [10]:
환승역 = subway[subway['환승역']==1]['검색키워드'].tolist()
환승역

['합정역 6호선',
 '약수역 6호선',
 '청구역 6호선',
 '신당역 6호선',
 '군자역 7호선',
 '강남구청역 7호선',
 '종로3가역 5호선',
 '올림픽공원역 5호선',
 '불광역 6호선',
 '연신내역 6호선',
 '디지털미디어시티역 6호선',
 '고속터미널역 7호선',
 '양재역 3호선',
 '도곡역 3호선',
 '수서역 3호선',
 '김포공항역 5호선',
 '잠실역 2호선',
 '옥수역 3호선',
 '고속터미널역 3호선',
 '교대역 3호선',
 '천호역 8호선',
 '종합운동장역 2호선',
 '선릉역 2호선',
 '강남역 2호선',
 '교대역 2호선',
 '종로3가역 3호선',
 '을지로3가역 3호선',
 '공덕역 6호선',
 '효창공원앞역 6호선',
 '석촌역 9호선',
 '올림픽공원역 9호선',
 '사당역 2호선',
 '신도림역 2호선',
 '영등포구청역 2호선',
 '삼각지역 6호선',
 '연신내역 3호선',
 '불광역 3호선',
 '동묘앞역 6호선',
 '을지로3가역 2호선',
 '을지로4가역 2호선',
 '동대문역사문화공원역 2호선',
 '신당역 2호선',
 '건대입구역 2호선',
 '당산역 2호선',
 '합정역 2호선',
 '홍대입구역 2호선',
 '충정로역 2호선',
 '충정로역 5호선',
 '대림역 7호선',
 '가산디지털단지역 7호선',
 '온수역 7호선',
 '가락시장역 3호선',
 '오금역 3호선',
 '노원역 4호선',
 '창동역 4호선',
 '성신여대입구역 4호선',
 '동대문역 4호선',
 '동대문역사문화공원역 4호선',
 '왕십리역 5호선',
 '보문역 6호선',
 '석계역 6호선',
 '태릉입구역 6호선',
 '상봉역 7호선',
 '까치산역 5호선',
 '영등포구청역 5호선',
 '신길역 5호선',
 '여의도역 5호선',
 '공덕역 5호선',
 '을지로4가역 5호선',
 '군자역 5호선',
 '천호역 5호선',
 '오금역 5호선',
 '잠실역 8호선',
 '선정릉역 9호선',
 '종합운동장역 9호선',
 '건대

In [11]:
for i in tqdm(range(len(data))):
    for j in 환승역:
        if j in data['전철역명(500m)'][i]:
            data['환승역'][i] = 1

100%|███████████████████████████████████████████████████████████████████████████| 12342/12342 [00:47<00:00, 260.89it/s]


In [12]:
data[data['환승역']==1]

,도로명,위도,경도,초역세권/역세권,전철역명(500m),환승역
45,선릉로125길,37.514731,127.040595,2,"[강남구청역 7호선, 강남구청역 분당선]",1
47,도산대로54길,37.521006,127.038931,1,"[강남구청역 7호선, 강남구청역 분당선]",1
51,선릉로137길,37.518990,127.039176,2,"[강남구청역 7호선, 강남구청역 분당선]",1
53,봉은사로51길,37.511719,127.042375,2,"[선정릉역 9호선, 선정릉역 분당선]",1
54,선릉로115길,37.511867,127.041155,2,"[선정릉역 9호선, 선정릉역 분당선]",1
...,...,...,...,...,...,...
12314,백범로37길,37.543324,126.954664,2,"[공덕역 6호선, 공덕역 5호선, 공덕역 경의선, 공덕역 공항철도]",1
12319,서초대로45길,37.493353,127.009148,2,"[교대역 3호선, 교대역 2호선, 서초역 2호선]",1
12337,종로46길,37.570724,127.010477,2,"[동묘앞역 6호선, 동대문역 4호선, 동대문역 1호선]",1
12339,종로35가길,37.571897,127.004875,2,"[동대문역 4호선, 종로5가역 1호선]",1


In [13]:
print(data.shape)
data.head()

(12342, 6)


,도로명,위도,경도,초역세권/역세권,전철역명(500m),환승역
0,논현로6길,37.473561,127.050895,0,[],0
1,논현로18길,37.477885,127.046338,0,[],0
2,개포로20길,37.477040,127.048356,0,[],0
3,선릉로12길,37.483899,127.060991,1,[구룡역 분당선],0
4,논현로8길,37.474467,127.051458,0,[],0


In [14]:
data.to_csv('data/단독다가구_지하철추가.csv', index=False)

## 2. 연립

In [15]:
data2 = pd.read_csv('data/연립_위도경도.csv')
print(data2.shape)
data2.head()

(82435, 3)


,도로명,위도,경도
0,논현로 10,37.472388,127.050511
1,논현로2길 66,37.474544,127.055084
2,논현로8길 62-1,37.474544,127.055084
3,논현로8길 63,37.475313,127.053546
4,개포로26길 16,37.478034,127.049109


### 2.1 초역세권/역세권, 전철역명(500m) 컬럼 추가

In [16]:
data2['초역세권/역세권'] = 0
data2['전철역명(500m)'] = 0

In [17]:
for i in tqdm(range(len(data2))):
    time.sleep(0.1)
    list2 = []
    for j in range(len(subway)):
        start = (float(data2['위도'][i]), float(data2['경도'][i]))
        goal = (float(subway['위도'][j]), float(subway['경도'][j]))
        distance = haversine(start, goal, unit='m')
        if distance <= 500:
            list2.append(subway['검색키워드'][j])
            if distance <= 300:
                data2['초역세권/역세권'][i] = 2
            elif data2['초역세권/역세권'][i] != 2:
                data2['초역세권/역세권'][i] = 1
    data2['전철역명(500m)'][i] = np.array(list2, dtype=str)

100%|██████████████████████████████████████████████████████████████████████████| 82435/82435 [3:02:18<00:00,  7.54it/s]


In [18]:
data2['초역세권/역세권'].value_counts()

0    41402
1    26506
2    14527
Name: 초역세권/역세권, dtype: int64

In [19]:
data2['전철역명(500m)']

0                                          []
1                                          []
2                                          []
3                                          []
4                                          []
                         ...                 
82430                               [중화역 7호선]
82431                               [중화역 7호선]
82432    [중화역 7호선, 상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]
82433                               [중화역 7호선]
82434                               [중화역 7호선]
Name: 전철역명(500m), Length: 82435, dtype: object

### 2.2 환승역 컬럼 추가

In [20]:
data2['환승역'] = 0

In [21]:
환승역 = subway[subway['환승역']==1]['검색키워드'].tolist()
환승역

['합정역 6호선',
 '약수역 6호선',
 '청구역 6호선',
 '신당역 6호선',
 '군자역 7호선',
 '강남구청역 7호선',
 '종로3가역 5호선',
 '올림픽공원역 5호선',
 '불광역 6호선',
 '연신내역 6호선',
 '디지털미디어시티역 6호선',
 '고속터미널역 7호선',
 '양재역 3호선',
 '도곡역 3호선',
 '수서역 3호선',
 '김포공항역 5호선',
 '잠실역 2호선',
 '옥수역 3호선',
 '고속터미널역 3호선',
 '교대역 3호선',
 '천호역 8호선',
 '종합운동장역 2호선',
 '선릉역 2호선',
 '강남역 2호선',
 '교대역 2호선',
 '종로3가역 3호선',
 '을지로3가역 3호선',
 '공덕역 6호선',
 '효창공원앞역 6호선',
 '석촌역 9호선',
 '올림픽공원역 9호선',
 '사당역 2호선',
 '신도림역 2호선',
 '영등포구청역 2호선',
 '삼각지역 6호선',
 '연신내역 3호선',
 '불광역 3호선',
 '동묘앞역 6호선',
 '을지로3가역 2호선',
 '을지로4가역 2호선',
 '동대문역사문화공원역 2호선',
 '신당역 2호선',
 '건대입구역 2호선',
 '당산역 2호선',
 '합정역 2호선',
 '홍대입구역 2호선',
 '충정로역 2호선',
 '충정로역 5호선',
 '대림역 7호선',
 '가산디지털단지역 7호선',
 '온수역 7호선',
 '가락시장역 3호선',
 '오금역 3호선',
 '노원역 4호선',
 '창동역 4호선',
 '성신여대입구역 4호선',
 '동대문역 4호선',
 '동대문역사문화공원역 4호선',
 '왕십리역 5호선',
 '보문역 6호선',
 '석계역 6호선',
 '태릉입구역 6호선',
 '상봉역 7호선',
 '까치산역 5호선',
 '영등포구청역 5호선',
 '신길역 5호선',
 '여의도역 5호선',
 '공덕역 5호선',
 '을지로4가역 5호선',
 '군자역 5호선',
 '천호역 5호선',
 '오금역 5호선',
 '잠실역 8호선',
 '선정릉역 9호선',
 '종합운동장역 9호선',
 '건대

In [22]:
for i in tqdm(range(len(data2))):
    for j in 환승역:
        if j in data2['전철역명(500m)'][i]:
            data2['환승역'][i] = 1

100%|███████████████████████████████████████████████████████████████████████████| 82435/82435 [05:32<00:00, 247.94it/s]


In [23]:
data2[data2['환승역']==1]

,도로명,위도,경도,초역세권/역세권,전철역명(500m),환승역
251,언주로138길 14,37.517766,127.036714,1,"[강남구청역 7호선, 강남구청역 분당선]",1
252,언주로136길 33,37.517898,127.038169,2,"[강남구청역 7호선, 강남구청역 분당선]",1
253,학동로43길 26,37.517816,127.037742,1,"[강남구청역 7호선, 강남구청역 분당선]",1
254,학동로53길 27,37.518855,127.039599,2,"[강남구청역 7호선, 강남구청역 분당선]",1
256,학동로47길 13-5,37.517327,127.038191,2,"[강남구청역 7호선, 강남구청역 분당선]",1
...,...,...,...,...,...,...
82410,면목로85길 20,37.593185,127.085088,1,"[상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]",1
82411,중랑천로12길 4,37.592185,127.073101,1,"[중랑역 경의선, 중랑역 경춘선]",1
82419,중랑역로18길 20-8,37.596905,127.077288,2,"[중랑역 경의선, 중랑역 경춘선]",1
82429,봉화산로4길 71,37.598597,127.074939,1,"[중랑역 경의선, 중랑역 경춘선]",1


In [24]:
print(data2.shape)
data2.head()

(82435, 6)


,도로명,위도,경도,초역세권/역세권,전철역명(500m),환승역
0,논현로 10,37.472388,127.050511,0,[],0
1,논현로2길 66,37.474544,127.055084,0,[],0
2,논현로8길 62-1,37.474544,127.055084,0,[],0
3,논현로8길 63,37.475313,127.053546,0,[],0
4,개포로26길 16,37.478034,127.049109,0,[],0


In [25]:
data2.to_csv('연립_지하철추가.csv', index=False)

## 3. 오피스텔

In [26]:
data3 = pd.read_csv('data/오피스텔_위도경도.csv')
print(data3.shape)
data3.head()

(3462, 3)


,도로명,위도,경도
0,개포로 213,37.478745,127.046113
1,개포로 623,37.494341,127.079648
2,논현로18길 10-10,37.477252,127.046084
3,선릉로129길 3-4,37.516104,127.041071
4,도산대로46길 11,37.521292,127.036652


### 3.1 초역세권/역세권, 전철역명(500m) 컬럼 추가

In [27]:
data3['초역세권/역세권'] = 0
data3['전철역명(500m)'] = 0

In [28]:
for i in tqdm(range(len(data3))):
    time.sleep(0.1)
    list3 = []
    for j in range(len(subway)):
        start = (float(data3['위도'][i]), float(data3['경도'][i]))
        goal = (float(subway['위도'][j]), float(subway['경도'][j]))
        distance = haversine(start, goal, unit='m')
        if distance <= 500:
            list3.append(subway['검색키워드'][j])
            if distance <= 300:
                data3['초역세권/역세권'][i] = 2
            elif data3['초역세권/역세권'][i] != 2:
                data3['초역세권/역세권'][i] = 1
    data3['전철역명(500m)'][i] = np.array(list3, dtype=str)

100%|██████████████████████████████████████████████████████████████████████████████| 3462/3462 [07:18<00:00,  7.90it/s]


In [35]:
data3['초역세권/역세권'].value_counts()

2    1310
0    1124
1    1028
Name: 초역세권/역세권, dtype: int64

In [36]:
data3['전철역명(500m)']

0                                []
1                         [대청역 3호선]
2                                []
3            [강남구청역 7호선, 강남구청역 분당선]
4                                []
                   ...             
3457    [상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]
3458    [상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]
3459                             []
3460             [중랑역 경의선, 중랑역 경춘선]
3461                      [중화역 7호선]
Name: 전철역명(500m), Length: 3462, dtype: object

### 3.2 환승역 컬럼 추가

In [29]:
data3['환승역'] = 0

In [30]:
환승역 = subway[subway['환승역']==1]['검색키워드'].tolist()
환승역

['합정역 6호선',
 '약수역 6호선',
 '청구역 6호선',
 '신당역 6호선',
 '군자역 7호선',
 '강남구청역 7호선',
 '종로3가역 5호선',
 '올림픽공원역 5호선',
 '불광역 6호선',
 '연신내역 6호선',
 '디지털미디어시티역 6호선',
 '고속터미널역 7호선',
 '양재역 3호선',
 '도곡역 3호선',
 '수서역 3호선',
 '김포공항역 5호선',
 '잠실역 2호선',
 '옥수역 3호선',
 '고속터미널역 3호선',
 '교대역 3호선',
 '천호역 8호선',
 '종합운동장역 2호선',
 '선릉역 2호선',
 '강남역 2호선',
 '교대역 2호선',
 '종로3가역 3호선',
 '을지로3가역 3호선',
 '공덕역 6호선',
 '효창공원앞역 6호선',
 '석촌역 9호선',
 '올림픽공원역 9호선',
 '사당역 2호선',
 '신도림역 2호선',
 '영등포구청역 2호선',
 '삼각지역 6호선',
 '연신내역 3호선',
 '불광역 3호선',
 '동묘앞역 6호선',
 '을지로3가역 2호선',
 '을지로4가역 2호선',
 '동대문역사문화공원역 2호선',
 '신당역 2호선',
 '건대입구역 2호선',
 '당산역 2호선',
 '합정역 2호선',
 '홍대입구역 2호선',
 '충정로역 2호선',
 '충정로역 5호선',
 '대림역 7호선',
 '가산디지털단지역 7호선',
 '온수역 7호선',
 '가락시장역 3호선',
 '오금역 3호선',
 '노원역 4호선',
 '창동역 4호선',
 '성신여대입구역 4호선',
 '동대문역 4호선',
 '동대문역사문화공원역 4호선',
 '왕십리역 5호선',
 '보문역 6호선',
 '석계역 6호선',
 '태릉입구역 6호선',
 '상봉역 7호선',
 '까치산역 5호선',
 '영등포구청역 5호선',
 '신길역 5호선',
 '여의도역 5호선',
 '공덕역 5호선',
 '을지로4가역 5호선',
 '군자역 5호선',
 '천호역 5호선',
 '오금역 5호선',
 '잠실역 8호선',
 '선정릉역 9호선',
 '종합운동장역 9호선',
 '건대

In [31]:
for i in tqdm(range(len(data3))):
    for j in 환승역:
        if j in data3['전철역명(500m)'][i]:
            data3['환승역'][i] = 1

100%|█████████████████████████████████████████████████████████████████████████████| 3462/3462 [00:13<00:00, 260.59it/s]


In [32]:
data3[data3['환승역']==1]

,도로명,위도,경도,초역세권/역세권,전철역명(500m),환승역
3,선릉로129길 3-4,37.516104,127.041071,2,"[강남구청역 7호선, 강남구청역 분당선]",1
5,학동로 342,37.516756,127.040841,2,"[강남구청역 7호선, 강남구청역 분당선]",1
6,학동로 338,37.516540,127.040162,2,"[강남구청역 7호선, 강남구청역 분당선]",1
9,선릉로129길 3-6,37.515962,127.041052,2,"[강남구청역 7호선, 강남구청역 분당선]",1
12,봉은사로 307,37.509552,127.040512,1,"[선정릉역 9호선, 선정릉역 분당선]",1
...,...,...,...,...,...,...
3455,면목로92길 28,37.595203,127.087416,2,"[상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]",1
3456,봉우재로41길 12,37.594040,127.089627,1,"[상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]",1
3457,봉우재로43길 11,37.594051,127.089925,1,"[상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]",1
3458,면목로94길 9,37.595433,127.086397,2,"[상봉역 7호선, 상봉역 경의선, 상봉역 경춘선]",1


In [33]:
print(data3.shape)
data3.head()

(3462, 6)


,도로명,위도,경도,초역세권/역세권,전철역명(500m),환승역
0,개포로 213,37.478745,127.046113,0,[],0
1,개포로 623,37.494341,127.079648,2,[대청역 3호선],0
2,논현로18길 10-10,37.477252,127.046084,0,[],0
3,선릉로129길 3-4,37.516104,127.041071,2,"[강남구청역 7호선, 강남구청역 분당선]",1
4,도산대로46길 11,37.521292,127.036652,0,[],0


In [34]:
data3.to_csv('data/오피스텔_지하철추가.csv', index=False)